# Task 3

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold

from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import RidgeClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import OneHotEncoder

from itertools import permutations 

In [2]:
test_data = pd.read_csv('test.csv') #These are just features names (48000)
train_data = pd.read_csv('train.csv') #Says if something is active or not (112000)

num_train_data = len(train_data)

In [3]:
train_feature = np.array(train_data['Sequence'])
train_output = np.array(train_data['Active'])
train_feature_splitted = []

In [4]:
for i, feature in enumerate(train_feature):
    train_feature_splitted.append([])
    for j, s in enumerate(list(feature)):
        train_feature_splitted[i].append(ord(s)*(j+1))

train_feature_splitted = np.array(train_feature_splitted)
train_feature_transform = np.copy(train_feature_splitted)

In [5]:
unique_feature = []
for j in range(4):
    unique_feature.append(np.unique(train_feature_splitted[:,j]))
    
unique_feature = np.array(unique_feature)

idx_list = []

for j in range(4):
    for i in range(20):
        idx_list.append(np.sum(train_output[train_feature_splitted[:,j]==unique_feature[j][i]]))
        
idx_list = np.array(idx_list)

sorted_idx = np.argsort(idx_list.flatten())
sorted_feature = unique_feature.flatten()[sorted_idx]

train_feature_sorted = np.zeros_like(train_feature_splitted)

for i, feature in enumerate(train_feature):
    for j, s in enumerate(list(feature)):
        train_feature_sorted[i,j] = sorted_feature[np.argwhere(unique_feature.flatten()==train_feature_splitted[i,j])]
        
train_feature_sorted = np.array(train_feature_sorted, dtype=float)

In [6]:
for i, j in permutations([0,1,2,3], 2):
    train_feature_mult = np.array([(train_feature_sorted[:,i]+1000)*train_feature_sorted[:,j]]).T
    train_feature_mult = train_feature_mult/np.max(train_feature_mult)
    train_feature_transform = np.concatenate((train_feature_transform, train_feature_mult), axis=1)

In [7]:
# for i, j, k in permutations([0,1,2,3], 3):
#     train_feature_mult = np.array([(train_feature_sorted[:,i]+1000)*
#                                    (train_feature_sorted[:,j]+300)*
#                                    train_feature_sorted[:,k]]).T
#     train_feature_mult = train_feature_mult/np.max(train_feature_mult)
#     train_feature_transform = np.concatenate((train_feature_transform, train_feature_mult), axis=1)

In [8]:
np.shape(train_feature_transform)

(112000, 16)

In [9]:
train_feature_splitted = np.array(train_feature_transform, dtype=float)

ohe = OneHotEncoder(handle_unknown='ignore')

train_feature_ohe = ohe.fit_transform(train_feature_splitted)

print(np.shape(train_feature_ohe))

(112000, 4848)


In [10]:
# tsvd = TruncatedSVD(n_components=16, algorithm='arpack', random_state=None)
# train_feature_ohe_tsvd = tsvd.fit_transform(train_feature_ohe)

In [11]:
score_func = make_scorer(f1_score)

In [12]:
%%time
model = KNeighborsClassifier(n_neighbors=1, n_jobs=6)
scores = cross_val_score(model, train_feature_transform, train_output, cv=5, scoring=score_func)

print(scores)

[0.55038303 0.52637749 0.52513966 0.54577259 0.53068182]
CPU times: user 4.93 s, sys: 127 ms, total: 5.06 s
Wall time: 7.32 s


In [13]:
%%time
model = DecisionTreeClassifier(criterion='entropy', random_state=None, class_weight='balanced')
scores = cross_val_score(model, train_feature_ohe, train_output, cv=5, scoring=score_func)

print(scores)

[0.79368536 0.80474649 0.78478261 0.78828082 0.78690284]
CPU times: user 9.23 s, sys: 31.8 ms, total: 9.26 s
Wall time: 9.26 s


In [14]:
%%time
model = DecisionTreeClassifier(criterion='entropy', random_state=None, class_weight='balanced')
scores = cross_val_score(model, train_feature_transform, train_output, cv=5, scoring=score_func)

print(scores)

[0.72881356 0.73563218 0.71698113 0.76093469 0.75816993]
CPU times: user 3.4 s, sys: 11.9 ms, total: 3.41 s
Wall time: 3.43 s


In [15]:
%%time
model = RandomForestClassifier(n_estimators=10, criterion='entropy', n_jobs=6)
scores = cross_val_score(model, train_feature_transform, train_output, cv=5, scoring=score_func)

print(scores)

[0.75771504 0.75862069 0.74484536 0.77204442 0.7562701 ]
CPU times: user 933 ms, sys: 91.7 ms, total: 1.03 s
Wall time: 3.48 s


In [ ]:
%%time
model = RandomForestClassifier(n_estimators=10, criterion='entropy', n_jobs=6, class_weight='balanced')
scores = cross_val_score(model, train_feature_ohe, train_output, cv=5, scoring=score_func)

print(scores)

In [ ]:
kf = KFold(n_splits=5, shuffle=False, random_state=None)
model = DecisionTreeClassifier(criterion='entropy', random_state=None, class_weight='balanced')
model2 = KNeighborsClassifier(n_neighbors=1, n_jobs=6)
#model = DecisionTreeRegressor(random_state=None)
#model = AdaBoostClassifier(n_estimators=10)

for train_idx, test_idx in kf.split(train_feature_splitted):
    
    model.fit(train_feature_ohe[train_idx], train_output[train_idx])
    model2.fit(train_feature_transform[train_idx], train_output[train_idx])
    
    prediction = model.predict(train_feature_ohe[test_idx])
    prediction2 = model2.predict(train_feature_transform[test_idx])
    print(f1_score(train_output[test_idx], np.logical_or(prediction, prediction2)))